# Système Expert Médical

Ce notebook implémente un système expert médical qui :
1. Analyse les symptômes pour établir un diagnostic
2. Recommande des médicaments appropriés
3. Fournit des explications sur les décisions prises

In [1]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules
from sklearn.preprocessing import MultiLabelBinarizer
import json
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

## 1. Chargement et préparation des données

In [2]:
# Chargement des données
medical_records = pd.read_csv('clean_medical_records.csv')
medications = pd.read_csv('openfda_meds.csv')

# Conversion des symptômes en liste
medical_records['symptoms'] = medical_records['symptoms'].apply(eval)

# Création d'une matrice binaire pour les symptômes
mlb = MultiLabelBinarizer()
symptoms_matrix = pd.DataFrame(
    mlb.fit_transform(medical_records['symptoms']),
    columns=mlb.classes_,
    index=medical_records.index
)

## 2. Génération des règles d'association

In [3]:
def generate_association_rules(min_support=0.1, min_confidence=0.5):
    """Génère les règles d'association entre symptômes et diagnostics"""
    # Génération des itemsets fréquents
    frequent_itemsets = apriori(symptoms_matrix, min_support=min_support, use_colnames=True)
    
    # Génération des règles
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)
    
    return rules

# Génération des règles
rules = generate_association_rules()
print(f"Nombre de règles générées : {len(rules)}")

Nombre de règles générées : 6


## 3. Système de diagnostic

In [4]:
class MedicalExpertSystem:
    def __init__(self, rules: pd.DataFrame, medical_records: pd.DataFrame, medications: pd.DataFrame):
        self.rules = rules
        self.medical_records = medical_records
        self.medications = medications
        
    def diagnose(self, symptoms: List[str]) -> Tuple[str, float, str]:
        """
        Établit un diagnostic basé sur les symptômes fournis
        
        Args:
            symptoms: Liste des symptômes
            
        Returns:
            Tuple contenant (diagnostic, confiance, explication)
        """
        # Recherche des règles correspondant aux symptômes
        matching_rules = self.rules[
            self.rules['antecedents'].apply(lambda x: all(s in x for s in symptoms))
        ]
        
        if len(matching_rules) == 0:
            return None, 0.0, "Aucune règle ne correspond aux symptômes fournis"
        
        # Sélection de la règle avec la plus haute confiance
        best_rule = matching_rules.loc[matching_rules['confidence'].idxmax()]
        
        # Recherche du diagnostic correspondant
        diagnosis = self.medical_records[
            self.medical_records['symptoms'].apply(lambda x: all(s in x for s in symptoms))
        ]['diagnosis'].mode().iloc[0]
        
        # Génération de l'explication
        explanation = f"Diagnostic basé sur la présence des symptômes : {', '.join(symptoms)}. "
        explanation += f"Confiance : {best_rule['confidence']:.2%}"
        
        return diagnosis, best_rule['confidence'], explanation
    
    def recommend_medications(self, diagnosis: str) -> List[Dict]:
        """
        Recommande des médicaments pour un diagnostic donné
        
        Args:
            diagnosis: Le diagnostic établi
            
        Returns:
            Liste des médicaments recommandés avec leurs informations
        """
        # Recherche des médicaments pertinents
        relevant_meds = self.medications[
            self.medications['indications'].str.contains(diagnosis, case=False, na=False)
        ]
        
        if len(relevant_meds) == 0:
            return []
        
        # Formatage des résultats
        recommendations = []
        for _, med in relevant_meds.iterrows():
            recommendations.append({
                'name': med['brand_name'],
                'purpose': med['purpose'],
                'indications': med['indications']
            })
        
        return recommendations

# Création de l'instance du système expert
expert_system = MedicalExpertSystem(rules, medical_records, medications)

## 4. Tests du système

In [5]:
# Test avec quelques symptômes
test_symptoms = ['fièvre', 'toux', 'fatigue']
diagnosis, confidence, explanation = expert_system.diagnose(test_symptoms)
print(f"Diagnostic : {diagnosis}")
print(f"Confiance : {confidence:.2%}")
print(f"Explication : {explanation}")

# Recommandations de médicaments
if diagnosis:
    recommendations = expert_system.recommend_medications(diagnosis)
    print("\nMédicaments recommandés :")
    for med in recommendations:
        print(f"\n- {med['name']}")
        print(f"  But : {med['purpose']}")
        print(f"  Indications : {med['indications']}")

Diagnostic : None
Confiance : 0.00%
Explication : Aucune règle ne correspond aux symptômes fournis


## 5. Intégration avec l'API Flask

In [6]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route("/diagnose", methods=["POST"])
def diagnose():
    data = request.get_json()
    symptoms = data.get("symptoms", [])
    
    if not symptoms:
        return jsonify({"error": "Aucun symptôme fourni"}), 400

    diagnosis, confidence, explanation = expert_system.diagnose(symptoms)
    
    if not diagnosis:
        return jsonify({"error": "Impossible d'établir un diagnostic"}), 404
    
    recommendations = expert_system.recommend_medications(diagnosis)
    
    return jsonify({
        "diagnosis": diagnosis,
        "confidence": confidence,
        "explanation": explanation,
        "recommendations": recommendations
    })

if __name__ == "__main__":
    app.run(debug=True) 

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/axel/Downloads/ownMade/notionsPractices/注記/2025 jan-mars(1st cycle) /project/fouille de données/eclat/expert system/venv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/axel/Downloads/ownMade/notionsPractices/注記/2025 jan-mars(1st cycle) /project/fouille de données/eclat/expert system/venv/lib/python3.12/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/home/axel/Downloads/ownMade/notionsPractices/注記/2025 jan-mars(1st cycle) /project/fouille de données/eclat/expert system/venv/lib/python3.12/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^

SystemExit: 1